# Proyecto final 2: Detección de SPAM con Machine Learning

### Enunciado y contexto del ejercicio

Se propone la construcción de un sistema de aprendizaje automático capaz de predecir si un correo determinado se corresponde con un correo de SPAM o no, para ello, se utilizará el conjunto de datos de Eron-Spam. El dataset contiene 33,719 correos electrónicos marcados como spam o como legítimos (ham). Los datos originales fueron recoletados por V. Metsis, I. Androutsopoulos y G. Paliouras. El conjunto de datos empaquetados se obtuvieron de este [repositorio](https://github.com/MWiechmann/enron_spam_data/tree/master).

| Archivo | Descripción |
| ----------- | --------------------------- |
| enron_spam_data.zip |  Es un archivo CSV comprimido que contiene las columnas `Message ID`, `subject`, `message`, `spam/ham` y la fecha.


### 1. Lectura de data

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('enron_spam_data.zip', compression='zip', index_col="Message ID")
data.head()

,Subject,Message,Spam/Ham,Date
Message ID,,,,
0,christmas tree farm pictures,NaN,ham,1999-12-10
1,"vastar resources , inc .","gary , production from the high island larger ...",ham,1999-12-13
2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,1999-12-14
3,re : issue,fyi - see note below - already done .\nstella\...,ham,1999-12-14
4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,1999-12-14


### 2. Procesamiento de lenguaje natural

**Pista 1**: Una vez que hayas eliminado todos los componentes del correo menos su cuerpo, explora la librería `nltk` para eliminar signos de puntuación y afijos. Revisa la clase `PorterStemmer()` de nltk así como los métodos y atributos `nltk.corpus.stopwords.words('english')`, `string.punctuation` y `nltk.tokenize.word_tokenize`

In [4]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation

stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))


In [5]:
filter_data = data.iloc[:20_000]

In [6]:
messages_tokens = []
statuses = []
for index, row in filter_data.iterrows():
    print("\r Processing message %d of %d" % (index + 1, len(filter_data)), end="")
    message = row['Message']
    if pd.notnull(message):
        message = message.lower()
        tokens = word_tokenize(message)
        tokens = [stemmer.stem(word) for word in tokens if word not in stop_words and word not in punctuation]
        messages_tokens.append(' '.join(tokens))
        statuses.append(row['Spam/Ham'])


 Processing message 20000 of 20000

In [8]:
statuses[:15_000].count('spam'), statuses[:15_000].count('ham')

(3001, 11999)

### 3. Verctorizando el conjunto de datos

Con las funciones presentadas anteriormente se permite la lectura de los correos electrónicos de manera programática y el procesamiento de los mismos para eliminar aquellos componentes que no resultan de utilidad para la detección de correos de SPAM. Sin embargo, cada uno de los correos sigue estando representado por un `string`.

La mayoría de los algoritmos de Machine Learning no son capaces de ingerir texto como parte del conjunto de datos. Por lo tanto, deben aplicarse una serie de funciones adicionales que transformen el texto de los correos electrónicos parseados en una representación numérica.

**Pista**: Revisa la clase `CountVectorizer` de Sklearn para realizar la codificación.

In [9]:
tokens_train, tokens_test = messages_tokens[:15_000], messages_tokens[15_001:]
y_train, y_test = statuses[:15_000], statuses[15_001:]

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(tokens_train)

print("Vocabulario: ", vectorizer.get_feature_names_out())
print("Matriz de características: ", X_train.toarray())

Vocabulario:  ['00' '000' '0000' ... 'þtien' 'þu' 'þöyledir']
Matriz de características:  [[ 2 18  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


### 4. Entrenamiento del algoritmo 

En este apartado vamos a entrenar un algoritmo de Machine Learning que aprenderá de los vectores anteriores a clasificar los correos en spam y legítimos.

Entrenar el algoritmo `LogisticRegression` de Sklearn. Este algoritmo se basa en aprendizaje supervisado y funciona exactamente igual que el algoritmo `Perceptron` que hemos presentado en secciones anteriores.

In [11]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression() # clasificador
clf.fit(X_train, y_train) # entrenamiento

LogisticRegression()

### 5. Predicción

Ya tenemos nuestro algoritmo entrenado y listo para realizar predicciones, lo único que nos queda es probar que tal se comporta para correos que no ha visto nunca.

Al aplicar la vectorización se utiliza únicamente el método `transform()` del objeto `CountVectorizer`.

In [16]:
from sklearn.metrics import accuracy_score, classification_report
X_test = vectorizer.transform(tokens_test)

In [17]:
y_pred = clf.predict(X_test)

print("Predicciones: ", y_pred)

Predicciones:  ['spam' 'spam' 'ham' ... 'spam' 'spam' 'spam']


In [18]:
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.9521601685985248
Classification Report:
              precision    recall  f1-score   support

         ham       0.89      0.97      0.93      1500
        spam       0.98      0.95      0.96      3245

    accuracy                           0.95      4745
   macro avg       0.94      0.96      0.95      4745
weighted avg       0.95      0.95      0.95      4745

